This notebook explores a mock dataset of stars, constellations, galaxies, and users. It demonstrates how I approach data analysis: cleaning, descriptive stats, inferential tests, and visualization — even in a science-inspired domain. Unlike my other projects, this project is focused on **statistics and analytical depth**.

This notebook will focus on **data loading and cleanup**.

First, load the CSVs into dataframes and display a few rows 

In [9]:
import pandas as pd


RAW_FOLDER = "../data/raw"
FLAGGED_FOLDER = "../data/flagged"
CLEANED_FOLDER = "../data/cleaned"

datasets = [
    ["users", None],
    ["galaxies", None],
    ["constellations", None],
    ["stars", None]
    ]

for i, [table, _] in enumerate(datasets):
    df = pd.read_csv(f"{RAW_FOLDER}/{table}.csv")
    print(f"\n=== Table: {table} (shape: {df.shape}) ===")
    display(df.head(3))
    # save the dataframe back into the list, for later use
    datasets[i][1] = df


=== Table: users (shape: (120, 7)) ===


,user_id,username,email,password,first_name,last_name,date_of_birth
0,1,taylorkim,taylorkim@example.com,REDACTED,Taylor,Kim,1997-01-12
1,2,rileygray,rileygray@example.com,REDACTED,Riley,Gray,1997-11-28
2,3,jamiejames,jamiejames@example.com,REDACTED,Jamie,James,2000-03-18



=== Table: galaxies (shape: (417, 9)) ===


,galaxy_id,galaxy_name,galaxy_type,distance_mly,redshift,mass_solar,diameter_ly,added_by,verified_by
0,1,Galaxy-001,Spiral,317.044,58.0326,275596326285,114330,23.0,113.0
1,2,Galaxy-002,Irregular,6.726,0.486,30715124530,10575,16.0,101.0
2,3,Galaxy-003,Spiral,3.460,3.0099,99131106087,82716,41.0,111.0



=== Table: constellations (shape: (418, 5)) ===


,constellation_id,constellation_name,galaxy_id,added_by,verified_by
0,1,Constellation-001,232,1,97
1,2,Constellation-002,341,18,98
2,3,Constellation-003,397,71,115



=== Table: stars (shape: (518, 12)) ===


,star_id,star_name,star_type,constellation_id,right_ascension,declination,apparent_magnitude,spectral_type,distance_mly,temperature_k,added_by,verified_by
0,1,Star-00001,Giant,5,245.746,-64.486,3.705,M,577.651,2887.0,89.0,107.0
1,2,Star-00002,Main Sequence,362,318.097,-83.532,3.085,G,312.768,5608.0,77.0,110.0
2,3,Star-00003,Main Sequence,317,226.618,-26.443,1.641,G,26.122,5749.0,34.0,99.0


In real projects, not all “messy” data should be cleaned automatically.

- Some issues are safe to fix (e.g., casing, whitespace, duplicate rows).
- Others, like potential typos or suspicious values, can’t be corrected without a reference list or domain knowledge.

In this notebook, I’ll demonstrate both:

- Cleaning where it’s safe,
- And flagging suspect values that would normally be escalated to a data owner.

The raw CSVs include ~10–20 intentionally messy rows per table (stars, constellations, galaxies).  
These simulate realistic data quality issues an analyst would face when pulling directly from a source system.  

Mess types added:
- **Typos / case mismatches** (e.g., "milky way" vs "Milky Way")  
- **Out-of-range values** (e.g., negative distances, impossible magnitudes)  
- **Missing values (NaN/blank fields)**  
- **Broken references** (e.g., star points to a non-existent constellation_id)  

Note: I do **not** attempt to “perfect” every typo or naming inconsistency (e.g., spelling of galaxy names).  
The goal is to show how to detect, flag, and handle common mess — not to endlessly hand-curate data.  

For these datasets, the business rules are as follows:

- There must not be **any** blank fields.
- There must not be any duplicate records.
- Names of users, galaxies, constellations and stars must not have any leading nor trailing spaces.
- Names must be in the proper case i.e. uppercase for first letters of words, lowercase for the rest.
- Values for `added_by` and `verified_by` must exist in **users**.`user_id`.
- Values for **stars**.`constellation_id` must exist in **constellations**.`constellation_id`.
- Values for **constellations**.`galaxy_id` must exist in **galaxies**.`galaxy_id`.
- In **stars**, `apparent_magnitude` must be between 0 and 10; `distance_mly` must be between 0 and 1000.
- In **galaxies**, `redshift` must be between 0 and 180; `distance_mly` must be between 0 and 1000.

In [10]:
# Do the straightforward cleanup steps first
for i, [_, df] in enumerate(datasets):
    if i == 0:
        # no need to modify the **users** dataframe
        continue
    df = df.map(lambda v: v.strip() if isinstance(v, str) else v)  # trim spaces
    df = df.map(lambda v: v.title() if isinstance(v, str) else v)  # fix case issues
    columns_without_pk = list(df)[1:]
    df = df.drop_duplicates(subset=columns_without_pk)  # remove duplicates
    df = df.fillna("")
    # save the dataframe back into the list, for later use
    datasets[i][1] = df

In [11]:
# Do dataframe-specific cleanup

df = datasets[0][1]
# no cleanup needed for **users** so just retrieve the user_ids
set_of_user_ids = set(df["user_id"])
df["remarks"] = "ALL GOOD"
datasets[0][1] = df

# for all other dataframes:
# create an empty list, to note down remarks
# iterate through rows.
# for each row:
#   analyze each field in the row for issues and flag them
#   if there are no issues, mark as good
#   append the list of issues to the list of remarks
# append the remarks to the dataframe as a new column


In [12]:
# for galaxies
df = datasets[1][1]
list_of_remarks = []
set_of_galaxy_ids = set()
df_columns = list(df)

for row in df.itertuples():
    row_remarks = []
    for column in df_columns:
        if getattr(row, column) == "":
            row_remarks.append(f"Value missing for {column}")

    try:
        if row.redshift and not 0 <= float(row.redshift) <= 180:
            row_remarks.append("Invalid value for redshift")
        if row.distance_mly and not 0 <= float(row.distance_mly) <= 1000:
            row_remarks.append("Invalid value for distance_mly")
        if row.added_by and int(row.added_by) not in set_of_user_ids:
            row_remarks.append("Invalid value for added_by")
        if row.verified_by and int(row.verified_by) not in set_of_user_ids:
            row_remarks.append("Invalid value for verified_by")
        if len(row_remarks) == 0:
            row_remarks.append("ALL GOOD")
            set_of_galaxy_ids.add(row.galaxy_id)
    except Exception as e:
        row_remarks.append(str(e))

    list_of_remarks.append("; ".join(row_remarks))

df["remarks"] = list_of_remarks
datasets[1][1] = df

In [13]:
# for df_constellations
df = datasets[2][1]
list_of_remarks = []
set_of_constellation_ids = set()
df_columns = list(df)

for row in df.itertuples():
    row_remarks = []
    for column in df_columns:
        if getattr(row, column) == "":
            row_remarks.append(f"Value missing for {column}")

    try:
        if row.galaxy_id and int(row.galaxy_id) not in set_of_galaxy_ids:
            row_remarks.append("Invalid value for galaxy_id")
        if row.added_by and int(row.added_by) not in set_of_user_ids:
            row_remarks.append("Invalid value for added_by")
        if row.verified_by and int(row.verified_by) not in set_of_user_ids:
            row_remarks.append("Invalid value for verified_by")
        if len(row_remarks) == 0:
            row_remarks.append("ALL GOOD")
            set_of_constellation_ids.add(row.constellation_id)
    except Exception as e:
        row_remarks.append(str(e))

    list_of_remarks.append("; ".join(row_remarks))

df["remarks"] = list_of_remarks
datasets[2][1] = df

In [14]:
# for stars
df = datasets[3][1]
list_of_remarks = []
df_columns = list(df)

for row in df.itertuples():
    row_remarks = []
    for column in df_columns:
        if getattr(row, column) == "":
            row_remarks.append(f"Value missing for {column}")

    try:
        if row.apparent_magnitude and not 0 <= float(row.apparent_magnitude) <= 10:
            row_remarks.append("Invalid value for apparent_magnitude")
        if row.distance_mly and not 0 <= float(row.distance_mly) <= 1000:
            row_remarks.append("Invalid value for distance_mly")
        if row.constellation_id and int(row.constellation_id) not in set_of_constellation_ids:
            row_remarks.append("Invalid value for constellation_id")
        if row.added_by and int(row.added_by) not in set_of_user_ids:
            row_remarks.append("Invalid value for added_by")
        if row.verified_by and int(row.verified_by) not in set_of_user_ids:
            row_remarks.append("Invalid value for verified_by")
        if len(row_remarks) == 0:
            row_remarks.append("ALL GOOD")
    except Exception as e:
        row_remarks.append(str(e))

    list_of_remarks.append("; ".join(row_remarks))

df["remarks"] = list_of_remarks
datasets[3][1] = df

This split approach reflects analyst practice:

- Safe fixes (like casing, whitespace, duplicates) are applied directly.
- Uncertain cases (like possible typos or implausible values) are flagged, not “magically fixed.”

This avoids introducing errors while still delivering clean, usable datasets for analysis.

Now write the flagged and cleaned dataframes into new CSV files.

In [15]:
for table, df in datasets:
    df[df["remarks"] != "ALL GOOD"].to_csv(f"{FLAGGED_FOLDER}/{table}.csv", index=False)
    df[df["remarks"] == "ALL GOOD"].drop(
        columns=["remarks"]).to_csv(f"{CLEANED_FOLDER}/{table}.csv", index=False)